INICIALIZACIÓN DE LAS PRIMERAS LIBRERIAS (FALTA POR ACUMULAR TODAS LAS LIBREARÍAS AQUI)

In [1]:
import os
import numpy as np
#import gdal
from spectral.io import envi, spyfile

In [2]:
from osgeo import gdal
print(gdal.__version__)

3.6.2


In [10]:
pip install spectra

Note: you may need to restart the kernel to use updated packages.


CREAR EL ARREGLO DE TEXTO CON CADA URL DE LAS DESCARGAS DE UN ARCHIVO TXT OBTENIDO DESDE EL PORTAL https://ode.rsl.wustl.edu/mars/download

In [3]:
# Ruta al archivo que contiene las URLs (ajusta según tu caso)
ruta_archivo = r"C:\Users\Yael Eduardo\TESTS_DATAH\FASE 2\IMG_FILES\GALE_CRATER_Links.txt"  # Reemplaza con la ruta real

# Leer el contenido del archivo
with open(ruta_archivo, 'r') as f:
    urls = f.read().splitlines()

VERIFICACIÓN SI ES QUE LAS URLS EXISTEN EN EL DATAFRAME Y SE HAYA LEÍDO BIEN EL TXT

In [4]:
print("Lista de URLs encontradas:")
for i, url in enumerate(urls, start=1):
    print(f"{i}: {url}")

Lista de URLs encontradas:
1: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcarj_mtr3.hdr
2: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcarj_mtr3.img
3: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcarj_mtr3.lbl
4: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcarj_mtr3.png
5: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcarj_mtr3.xml
6: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brchlj_mtr3.hdr
7: https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/b

A PARTIR DE LA LISTA OBTENIDA, SE CLASIFICAN Y SE FILTRAN SOLAMENTE LOS ARCHIVOS ÚTILES (hdr, img y lbl) que serán los necesarios para realizar el procesamiento de datos

SE IMPRIMEN LOS PRIMEROS 5 ELEMENTOS PARA VERIFICAR SI ES QUE SI EXISTE

In [5]:
import pandas as pd

# Supongamos que ya tienes esta lista cargada
# urls = [...]

# Lista para almacenar las bases válidas
urls_base = []

# Filtramos las extensiones válidas
extensiones = ['.hdr', '.img', '.lbl']

# Creamos un conjunto con los nombres base de las URLs
nombres_base = {}

for url in urls:
    if any(url.endswith(ext) for ext in extensiones):
        base, ext = os.path.splitext(url)
        if base not in nombres_base:
            nombres_base[base] = set()
        nombres_base[base].add(ext)

# Revisamos cuáles tienen las 3 extensiones
for base, ext_set in nombres_base.items():
    if all(ext in ext_set for ext in extensiones):
        urls_base.append(base)

# Crear DataFrame
df = pd.DataFrame({'URL base': urls_base})

# Mostrar los primeros 5 resultados
print(df.head().to_string(index=False))


                                                                                                                                    URL base
https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcarj_mtr3
https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brchlj_mtr3
https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brcr2j_mtr3
https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brfalj_mtr3
https://pds-geosciences.wustl.edu/mro/mro-m-crism-5-rdr-mptargeted-v1/mrocr_4001/browse/2006/2006_319/hrl00003010/hrl00003010_07_brfemj_mtr3


CODIGO PARA PODER DESCARGAR LOS ARCHIVOS FILTRADOS Y SELECCIONADOS EN EL ANTERIOR DATAFRAME Y GUARDARLO EN SUBCARPETAS DEPENDIENDO EL TIPO DE MISIÓN Y LA CLASIFICACIÓN DEL ARCHIVO BASE DE PSD

In [ ]:
import os
import urllib.request

# Directorio de trabajo
BASE_DIR = r"D:\MARS\FASE 2"

# Lista de URLs base del DataFrame
directorios_a_procesar = df['URL base'].tolist()

def descargar_archivos_crism(url_completa, directorio_destino):
    """Descarga los archivos .lbl, .img y .hdr desde la URL completa especificada."""

    # Crear el directorio de destino si no existe
    os.makedirs(directorio_destino, exist_ok=True)

    # Obtener el nombre base del archivo a partir de la URL completa
    im_name = os.path.basename(url_completa)

    # Construir las rutas completas para los archivos de destino
    im_path_lbl = os.path.join(directorio_destino, f"{im_name}.lbl")
    im_path_img = os.path.join(directorio_destino, f"{im_name}.img")
    im_path_hdr = os.path.join(directorio_destino, f"{im_name}.hdr")

    # Descargar los archivos si no existen
    try:
        for extension, im_path in [(".lbl", im_path_lbl), (".img", im_path_img), (".hdr", im_path_hdr)]:
            if not os.path.isfile(im_path):
                print(f"Descargando {im_path}")
                urllib.request.urlretrieve(f"{url_completa}{extension}", im_path)
                print(f"Archivo {extension} guardado en {im_path}")
            else:
                print(f"El archivo {im_path} ya existe.")
    except Exception as e:
        print(f"Error al descargar los archivos de {url_completa}: {e}")

# Procesar cada URL completa en la lista
for url_completa in directorios_a_procesar:
    # Construir la ruta del directorio de destino a partir de la URL completa
    directorio_relativo = '/'.join(url_completa.split('/')[-4:]) 
    directorio_destino = os.path.join(BASE_DIR, directorio_relativo)

    # Descargar los archivos para esta URL
    descargar_archivos_crism(url_completa, directorio_destino)


MOVER LOS ARCHIVOS CREADOS EN SUBCARPETA A SOLAMENTE UNA (LOS NECESARIOS)

In [ ]:
import os
import shutil

# Directorio base donde se encuentran las subcarpetas con los archivos descargados
directorio_base = r"D:\MARS\FASE 2"

# Carpeta de destino donde quieres mover todos los archivos
carpeta_destino = r"D:\MARS\NEW" 

# Crear la carpeta de destino si no existe
os.makedirs(carpeta_destino, exist_ok=True)

# Función para recorrer recursivamente las subcarpetas y mover los archivos
def mover_archivos(ruta_actual):
    for elemento in os.listdir(ruta_actual):
        ruta_elemento = os.path.join(ruta_actual, elemento)
        
        if os.path.isfile(ruta_elemento):
            # Si es un archivo, moverlo a la carpeta de destino
            shutil.move(ruta_elemento, carpeta_destino)
            print(f"Movido: {ruta_elemento} -> {carpeta_destino}")
        elif os.path.isdir(ruta_elemento):
            # Si es una carpeta, llamar recursivamente a la función
            mover_archivos(ruta_elemento)

# Iniciar el proceso desde el directorio base
mover_archivos(directorio_base)

EJECUTAR PARA PODER CORRER LA CLASIFICACIÓN

In [3]:
def geospatial_info(fname):
    dataset = gdal.Open(fname)
    if dataset is None:
        raise FileNotFoundError(f"No se pudo abrir el archivo: {fname}")
    
    geotransform = dataset.GetGeoTransform()
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    return geotransform, width, height

def pixel_to_geo(x, y, geotransform):
    """Convierte las coordenadas de píxel (x, y) a coordenadas geográficas (lon, lat)."""
    lon = geotransform[0] + x * geotransform[1] + y * geotransform[2]
    lat = geotransform[3] + x * geotransform[4] + y * geotransform[5]
    return lon, lat

def calculate_geocoordinates(width, height, geotransform):
    """Calcula las coordenadas geográficas de cada píxel en la imagen."""
    xx_, yy_ = np.meshgrid(np.arange(width), np.arange(height))
    geo_coordinates = np.zeros((height, width, 2))  # Matriz para almacenar lon y lat

    for y in range(height):
        for x in range(width):
            lon, lat = pixel_to_geo(x, y, geotransform)
            geo_coordinates[y, x] = [lon, lat]

    Ldict = {
        'x': xx_.ravel() + 1,  # +1 para ajustar a base 1 si es necesario
        'y': yy_.ravel() + 1,
        'lon': geo_coordinates[:, :, 0].ravel(),
        'lat': geo_coordinates[:, :, 1].ravel()
    }
    return Ldict

def load_envi_data(fname, band_select):
    if not fname.endswith(('.img', '.hdr')):
        fname += '.img'

    fbase, ext = os.path.splitext(fname)
    hdr_fname = f"{fbase}.hdr"

    if not os.path.exists(hdr_fname):
        _generate_envi_header(f"{fbase}.lbl")

    try:
        img = envi.open(hdr_fname)
        arr = img.load()
    except (spyfile.FileNotFoundError, EOFError) as e:
        raise FileNotFoundError(f"Unable to load ENVI file: {fname}. Check if the file exists and is not corrupted.") from e

    # Detectar bandas disponibles y ajustar band_select
    max_bandas = arr.shape[2]
    band_select_filtered = [b for b in band_select if b < max_bandas]
    if not band_select_filtered:
        raise ValueError(f"Archivo {fname} no tiene bandas válidas para band_select.")

    mdict = {'IF': arr[:, :, band_select_filtered]}
    xx_, yy_ = np.meshgrid(np.arange(arr.shape[1]), np.arange(arr.shape[0]))
    mdict.update({'x': xx_.ravel() + 1, 'y': yy_.ravel() + 1})
    mdict['IF'] = mdict['IF'].reshape((-1, len(band_select_filtered)))

    return mdict

def create_pixel_to_coordinates_dict(mdict, Ldict):
    """Crea un diccionario que asocia cada píxel con sus coordenadas geográficas y valores IF."""
    pixel_to_coordinates = {}
    for idx in range(len(mdict['x'])):
        x_val = mdict['x'][idx]
        y_val = mdict['y'][idx]
        lon_val = Ldict['lon'][idx]
        lat_val = Ldict['lat'][idx]

        # Añadir la coordenada y el valor IF al diccionario
        pixel_to_coordinates[(x_val, y_val)] = {
            'lon': lon_val,
            'lat': lat_val,
            'IF': mdict['IF'][idx]
        }
    return pixel_to_coordinates

In [18]:
import os
# ... (otras importaciones necesarias, incluyendo las funciones que usas)

# Directorio donde se encuentran los archivos .img (reemplaza con la ruta correcta)
directorio_imagenes = r"D:\MARS\NEW"

# Obtener la lista de archivos .img en el directorio
archivos_img = [f for f in os.listdir(directorio_imagenes) if f.endswith('.img')]

# Listas para almacenar los resultados
archivos_procesados_ok = []
archivos_con_error = []

# Iterar sobre los archivos e intentar procesarlos
for archivo in archivos_img:
    ruta_archivo = os.path.join(directorio_imagenes, archivo)
    fname = ruta_archivo  # Asumiendo que 'im_name' es el nombre del archivo sin extensión

    try:
        # Tu código para procesar el archivo
        band_select = np.r_[433:185:-1, 170:-1:68]
        geotransform, width, height = geospatial_info(fname)
        Ldict = calculate_geocoordinates(width, height, geotransform)
        mdict = load_envi_data(fname, band_select)
        pixel_to_coordinates = create_pixel_to_coordinates_dict(mdict, Ldict)

        # Si el procesamiento llega hasta aquí sin errores, el archivo es OK
        archivos_procesados_ok.append(archivo)
        print(f"Archivo procesado OK: {ruta_archivo}")

    except Exception as e:
        # Si ocurre algún error durante el procesamiento, registrar el archivo y el error
        archivos_con_error.append((archivo, str(e)))
        print(f"Error al procesar el archivo: {ruta_archivo}. Error: {e}")

# Mostrar los resultados
print("\nArchivos procesados correctamente:")
for archivo in archivos_procesados_ok:
    print(archivo)

print("\nArchivos con errores:")
for archivo, error in archivos_con_error:
    print(f"{archivo}: {error}")

Error al procesar el archivo: D:\MARS\NEW\frt000037df_07_brcarj_mtr3.img. Error: Archivo D:\MARS\NEW\frt000037df_07_brcarj_mtr3.img no tiene bandas válidas para band_select.
Error al procesar el archivo: D:\MARS\NEW\frt000037df_07_brchlj_mtr3.img. Error: Archivo D:\MARS\NEW\frt000037df_07_brchlj_mtr3.img no tiene bandas válidas para band_select.
Error al procesar el archivo: D:\MARS\NEW\frt000037df_07_brcr2j_mtr3.img. Error: Archivo D:\MARS\NEW\frt000037df_07_brcr2j_mtr3.img no tiene bandas válidas para band_select.
Error al procesar el archivo: D:\MARS\NEW\frt000037df_07_brfalj_mtr3.img. Error: Archivo D:\MARS\NEW\frt000037df_07_brfalj_mtr3.img no tiene bandas válidas para band_select.
Error al procesar el archivo: D:\MARS\NEW\frt000037df_07_brfemj_mtr3.img. Error: Archivo D:\MARS\NEW\frt000037df_07_brfemj_mtr3.img no tiene bandas válidas para band_select.
Error al procesar el archivo: D:\MARS\NEW\frt000037df_07_brfm2j_mtr3.img. Error: Archivo D:\MARS\NEW\frt000037df_07_brfm2j_mtr3.im

KeyboardInterrupt: 

In [17]:
print("\n✅ Archivos procesados correctamente:")
for archivo in archivos_procesados_ok:
    print(f"  ✔ {archivo}")

print("\n❌ Archivos con errores:")
for archivo, error in archivos_con_error:
    print(f"  ✘ {archivo} → {error}")


✅ Archivos procesados correctamente:


NameError: name 'archivos_procesados_ok' is not defined

In [8]:
n = 5  # Número de elementos que quieres imprimir

# Usando un bucle:
for i, (key, value) in enumerate(pixel_to_coordinates.items()):
    if i >= n:
        break
    print(f"Clave: {key}, Valor: {value}")

# Usando itertools.islice para cortar el diccionario:
from itertools import islice

for key, value in islice(pixel_to_coordinates.items(), n):
    print(f"Clave: {key}, Valor: {value}")

Clave: (1, 1), Valor: {'lon': 8151395.9, 'lat': -322060.75, 'IF': array([65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       65535., 65535., 65535., 65535., 65535., 65535., 65535., 65535.,
       6553

In [21]:
from spectral import open_image

# Ruta al archivo de imagen
img_path = r"D:\MARS\NEW\frt000037df_07_if165j_mtr3.hdr"

# Cargar la imagen usando spectra
img = open_image(img_path)

# Obtener las longitudes de onda
wavelengths = img.metadata['wavelength']

#print(wavelengths)
print(len(wavelengths))
#print(wavelengths.index('1047.200000'))

FileNotFoundError: Unable to locate file "D:\MARS\NEW\fhrl00003010_07_if181j_mtr3.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(pixel_to_coordinates, orient='index')
print(df)

               lon        lat  \
1   1    8151395.9 -322060.75   
2   1    8151431.9 -322060.75   
3   1    8151467.9 -322060.75   
4   1    8151503.9 -322060.75   
5   1    8151539.9 -322060.75   
...            ...        ...   
324 255  8163023.9 -331204.75   
325 255  8163059.9 -331204.75   
326 255  8163095.9 -331204.75   
327 255  8163131.9 -331204.75   
328 255  8163167.9 -331204.75   

                                                        IF  
1   1    [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
2   1    [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
3   1    [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
4   1    [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
5   1    [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
...                                                    ...  
324 255  [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
325 255  [65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...  
326 255  [65535.0, 65535.0, 65535.0, 65535.0, 65535.0,

In [4]:
import csv

with open('pixel_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Escribir encabezado
    writer.writerow(['x', 'y', 'lon', 'lat', 'IF'])

    for (x, y), data in pixel_to_coordinates.items():
        row = [x, y, data['lon'], data['lat'], data['IF']]
        writer.writerow(row)


NameError: name 'pixel_to_coordinates' is not defined

In [4]:
def factors(im_shape):
    fact_h = []
    fact_w = []
    for i in range(1, im_shape[0] + 1):
        if im_shape[0] % i == 0:
            fact_h.append(i)
            
    for i in range(1, im_shape[1] + 1):
        if im_shape[1] % i == 0:
            fact_w.append(i)       
    return fact_h , fact_w


In [16]:
with open('archivos_ok.txt', 'w') as f:
    for archivo in archivos_procesados_ok:
        f.write(archivo + '\n')


NameError: name 'archivos_procesados_ok' is not defined

In [5]:
from crism_ml.io import load_image
from crism_ml.preprocessing import filter_bad_pixels
from osgeo import gdal
import matplotlib.pyplot as plt
from crism_ml.io import image_shape
from crism_ml.plot import get_false_colors
import numpy as np
from crism_ml.preprocessing import remove_spikes_column, replace, ratio
from crism_ml.train import compute_bland_scores
from crism_ml.preprocessing import remove_spikes
from crism_ml.train import compute_scores
from crism_ml.train import filter_predictions_2, evaluate_regions, merge_regions
from crism_ml.preprocessing import crop_region
from crism_ml.plot import show_classes

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import logging
logging.basicConfig(level=logging.INFO)

import os
import urllib.request

WORKDIR = "../workdir"
IMAGES = "../image_hdrs"
DATADIR = "../datasets"
os.makedirs(WORKDIR, exist_ok=True)
os.makedirs(IMAGES, exist_ok=True)
os.makedirs(DATADIR, exist_ok=True)

In [7]:
%%time
from crism_ml.train import feat_masks, train_model_bland, train_model, \
    iteration_weights


fin0, fin = feat_masks()
bmodels = train_model_bland(DATADIR, fin0)
models = train_model(DATADIR, fin)
ww_ = iteration_weights(models[0].classes)

INFO:root:Loading from cache: C:\Users\Yael Eduardo\TESTS_DATAH\IMPLEMENTATION\.cache\default_bmodel.pkl
INFO:root:Loading from cache: C:\Users\Yael Eduardo\TESTS_DATAH\IMPLEMENTATION\.cache\default_model.pkl


CPU times: total: 0 ns
Wall time: 78.7 ms


In [8]:
thresholds = (0.469,
    0.469,
    0.469,
    0.406,
    0.392,
    0.19,
    0.19,
    0.5,
    0.28,
    0.7,
    0.29,
    0.406,
    0.19,
    0.406,
    0.29,
    0.469,
    0.7,
    0.39,
    0.28,
    0.5,
    0.5,
    0.7,
    0.28,
    0.335,
    0.7,
    0.19,
    0.28,
    0.525,
    0.266,
    0.266,
    0.392,
    0.7,
    0.7,
)

print(len(thresholds)) #len must be 33

33


In [10]:
import os
import pandas as pd
import numpy as np

# === CONFIGURACIÓN ===
directorio_imagenes = r"D:\MARS\NEW"
archivo_lista_ok = 'archivos_ok.txt'  # archivo con los nombres válidos (uno por línea)

# === LEER LISTA DE ARCHIVOS OK ===
with open(archivo_lista_ok, 'r') as f:
    archivos_ok = [line.strip() for line in f.readlines()]

print(f"Se encontraron {len(archivos_ok)} archivos válidos para procesar.")

# === PROCESAR CADA ARCHIVO ===
for im_path in archivos_ok:
    ruta_completa = os.path.join(directorio_imagenes, im_path)
    archivo_h5 = f'{im_path}.h5'  # nombre del archivo h5 que se genera

    # ⏩ Verificar si el archivo .h5 ya existe
    if os.path.exists(archivo_h5):
        print(f"⏩ {archivo_h5} ya existe. Saltando procesamiento.")
        continue

    try:
        mat = load_image(ruta_completa)
        if_, rem = filter_bad_pixels(mat['IF'])

        fname = ruta_completa
        geotransform, width, height = geospatial_info(fname)
        shape_coordinates = calculate_geocoordinates(width, height, geotransform)

        im_shape = image_shape(mat)
        im_fc = get_false_colors(if_.reshape(*im_shape, -1), rem.reshape(im_shape))
        if1 = remove_spikes_column(if_.reshape(*im_shape, -1), 3, 5).reshape(if_.shape)
        slog = compute_bland_scores(if1, (bmodels, fin0))
        slog_inf = replace(slog, rem, -np.inf).reshape(im_shape)
        if2 = ratio(if1.reshape(*im_shape, -1), slog_inf, window=7).reshape(if_.shape)
        ifm = remove_spikes(if2.copy())
        sumlog = compute_scores(ifm, (models, fin), ww_)

        pred, pred0, pp_ = filter_predictions_2(sumlog, models[0].classes, kls_thr=thresholds)
        avgs = evaluate_regions(if2, im_shape, replace(pred, rem, 0), pp_, if0=if_)
        regs = merge_regions(avgs)

        pred_imshape = pred.reshape(im_shape)
        lat_imshape = shape_coordinates['lat'].reshape(im_shape)
        lon_imshape = shape_coordinates['lon'].reshape(im_shape)

        fact_height, fact_width = factors(im_shape)
        try:
            h_zone = fact_height[(len(fact_height) // 2 + 1)]
        except Exception:
            h_zone = fact_height[(len(fact_height) // 2 - 1)]

        try:
            w_zone = fact_width[(len(fact_width) // 2 + 1)]
        except Exception:
            w_zone = fact_width[(len(fact_width) // 2 - 1)]

        pred_zones = pred.reshape(h_zone, w_zone, -1)
        lon_zones = shape_coordinates['lon'].reshape(h_zone, w_zone, -1)
        lat_zones = shape_coordinates['lat'].reshape(h_zone, w_zone, -1)

        from crism_ml.lab import FULL_NAMES
        zones = pred_zones.shape[2]
        hist = np.zeros((zones, len(FULL_NAMES)))
        hist_clust = np.zeros((zones, len(FULL_NAMES) - 2))
        hist_clust_per = np.zeros((zones, len(FULL_NAMES) - 2))

        reshaped_pred = pred_zones.reshape(zones, h_zone * w_zone)

        for i in range(zones):
            hist_values, bins = np.histogram(
                reshaped_pred[i],
                bins=len(FULL_NAMES),
                range=(0, len(FULL_NAMES))
            )
            hist[i, :] = hist_values
            hist_clust[i, :] = np.delete(hist[i, :], [0, 38])
            hist_clust_per[i, :] = hist_clust[i, :] / np.sum(hist[i, :]) if np.sum(hist[i, :]) != 0 else 0

        im_file_to_clust = {
            'f_name': im_path,
            '_hist_clus_per': hist_clust_per,
            '_im_shape': im_shape,
            '_zone_shape': (h_zone, w_zone),
            '_predictions': pred_zones.reshape(h_zone * w_zone, zones),
            '_lat': lat_zones.reshape(h_zone * w_zone, zones),
            '_lon': lon_zones.reshape(h_zone * w_zone, zones)
        }

        with pd.HDFStore(archivo_h5, mode='w') as store:
            store.put('_hist_clus_per', pd.DataFrame(im_file_to_clust['_hist_clus_per']))
            store.put('_predictions', pd.DataFrame(im_file_to_clust['_predictions']))
            store.put('_lat', pd.DataFrame(im_file_to_clust['_lat']))
            store.put('_lon', pd.DataFrame(im_file_to_clust['_lon']))
            store.get_storer('_predictions').attrs._im_shape = im_file_to_clust['_im_shape']
            store.get_storer('_predictions').attrs._zone_shape = im_file_to_clust['_zone_shape']
            store.get_storer('_predictions').attrs.file_name = im_file_to_clust['f_name']

        print(f"✔ h5 file for {im_path} stored with success")

    except Exception as e:
        print(f"❌ Error processing {im_path}: {e}")


Se encontraron 51 archivos válidos para procesar.
⏩ frt000037df_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt000045f2_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt000058a3_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000901a_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt000095ee_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000a091_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000a906_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000b5a3_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000b6f1_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000bee7_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000bfca_07_if167j_mtr3.img.h5 ya existe. Saltando procesamiento.
⏩ frt0000c0ef_07_if165j_mtr3.img.h5 ya existe. Saltando procesamiento.


INFO:root:180898 bad pixels


753 782
✔ h5 file for frt0000c518_07_if165j_mtr3.img stored with success


INFO:root:243294 bad pixels


774 884
✔ h5 file for frt0000c620_07_if166j_mtr3.img stored with success


...

262 346
✔ h5 file for hrs0000326e_07_if174j_mtr3.img stored with success


INFO:root:23909 bad pixels


265 339
✔ h5 file for hrs0000336a_07_if174j_mtr3.img stored with success


INFO:root:22759 bad pixels


255 328
✔ h5 file for hrs00004259_07_if174j_mtr3.img stored with success
